In [1]:
import cv2
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from keras.layers import Dense, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam

2023-06-28 15:46:32.790202: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-28 15:46:32.792340: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 15:46:32.832295: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 15:46:32.832821: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 15:46:33.566852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
logs=[]
features=[]
labels=[]
with open('/home/tejveer/Desktop/Self Driving Car/data/driving_log.csv', 'rt') as f:
        rd= csv.reader(f)
        for line in rd:
            logs.append(line)
        log_labels = logs.pop(0)
for i in range(len(logs)):
        for j in range(3):
            img_path = logs[i][j]
            img_path = '/home/tejveer/Desktop/Self Driving Car/data/' + 'IMG' + (img_path.split('IMG')[1]).strip()
            img = plt.imread(img_path)
            resized_image = cv2.resize((cv2.cvtColor(img, cv2.COLOR_RGB2HSV))[:, :, 1], (40, 40))
            features.append(resized_image)
            if j == 0:
                labels.append(float(logs[i][3]))
            elif j == 1:
                labels.append(float(logs[i][3]) + 0.2)
            else:
                labels.append(float(logs[i][3]) - 0.2)
                
features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

In [3]:
model = Sequential()
model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(40, 40, 1)))

model.add(Conv2D(16, (3, 3), padding='same',activation='relu'))
model.add(Conv2D(16, (3, 3), padding='same',activation='relu'))
    
model.add(Conv2D(32, (3, 3), padding='same',activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same',activation='relu'))
model.add(MaxPooling2D((2, 2), padding='valid'))
    
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(MaxPooling2D((2, 2), padding='valid'))
    
model.add(Conv2D(128, (3, 3), padding='same',activation='relu'))
model.add(MaxPooling2D((2, 2), padding='valid'))

model.add(Conv2D(128, (3, 3), padding='same',activation='relu'))
model.add(MaxPooling2D((2, 2), padding='valid'))

model.add(Flatten())
model.add(Dropout(0.5))
    
model.add(BatchNormalization())
model.add(Dense(512))
model.add(Dense(256))
model.add(Dense(64))
model.add(Dense(1))
    
model.compile(optimizer=Adam(lr=0.0001), loss="mse")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 40, 40, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 40, 40, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 40, 40, 32)        4640      
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 20, 20, 32)       0         
 )                                                               
                                                        

/home/tejveer/anaconda3/lib/python3.9/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
features, labels = shuffle(features, labels)
train_x, val_x, train_y, val_y = train_test_split(features, labels, random_state=42, test_size=0.20)
train_x = train_x.reshape(train_x.shape[0], 40, 40, 1)
val_x = val_x.reshape(val_x.shape[0], 40, 40, 1)

In [5]:
filepath = "./SelfDriving.h5"
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=32)

model.save(filepath)

Epoch 1/10
603/603 [==============================] - 91s 150ms/step - loss: 0.0291 - val_loss: 0.0212
Epoch 2/10
603/603 [==============================] - 76s 126ms/step - loss: 0.0147 - val_loss: 0.0102
Epoch 3/10
603/603 [==============================] - 76s 126ms/step - loss: 0.0135 - val_loss: 0.0100
Epoch 4/10
603/603 [==============================] - 71s 118ms/step - loss: 0.0127 - val_loss: 0.0094
Epoch 5/10
603/603 [==============================] - 90s 150ms/step - loss: 0.0124 - val_loss: 0.0117
Epoch 6/10
603/603 [==============================] - 80s 133ms/step - loss: 0.0122 - val_loss: 0.0092
Epoch 7/10
603/603 [==============================] - 89s 148ms/step - loss: 0.0118 - val_loss: 0.0094
Epoch 8/10
603/603 [==============================] - 92s 152ms/step - loss: 0.0115 - val_loss: 0.0088
Epoch 9/10
603/603 [==============================] - 80s 132ms/step - loss: 0.0111 - val_loss: 0.0085
Epoch 10/10
603/603 [==============================] - 84s 139ms/step - l